In [10]:
import os
import requests
import pandas as pd

# Define your Alpha Vantage API key
ALPHA_VANTAGE_API_KEY = "ZUTBIAVI3SHXUGYY"

# Function to get the latest price for a given symbol
def get_latest_price(symbol):
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()
    latest_price = float(data['Global Quote']['05. price'])
    return latest_price

# Function to get historical daily prices for a given symbol and date range
def get_historical_prices(symbol, start_date, end_date):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()['Time Series (Daily)']
    
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    prices = []
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        if date_str in data:
            prices.append({
                'date': date_str,
                'open': float(data[date_str]['1. open']),
                'high': float(data[date_str]['2. high']),
                'low': float(data[date_str]['3. low']),
                'close': float(data[date_str]['4. close'])
            })
    
    return pd.DataFrame(prices)
    
def get_latest_price_realtime(symbol):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()
    latest_price = float(data['Time Series (1min)'][list(data['Time Series (1min)'].keys())[0]]['4. close'])
    return latest_price



if __name__ == "__main__":
    # (1) Get the latest price for VOO
    latest_voo_price = get_latest_price("VOO")
    print(f"Latest price for VOO: ${latest_voo_price:.2f}")
    latest_voo_price_realtime = get_latest_price_realtime("VOO")
    print(f"Latest price for VOO in real-time: ${latest_voo_price_realtime:.2f}")


    # (2) Get historical daily prices for VOO from 2024-01-01 to 2024-01-31
    historical_voo_prices = get_historical_prices("VOO", "2024-01-01", "2024-01-31")
    print("\nHistorical prices for VOO:")
    print(historical_voo_prices)

Latest price for VOO: $469.98
Latest price for VOO in real-time: $469.98

Historical prices for VOO:
          date    open      high       low   close
0   2024-01-02  433.91  435.2700  432.3600  434.01
1   2024-01-03  432.32  433.0000  430.2500  430.79
2   2024-01-04  430.33  432.7800  429.2100  429.43
3   2024-01-05  429.59  432.3000  428.6400  429.98
4   2024-01-08  430.45  436.2700  430.3700  436.13
5   2024-01-09  433.62  436.4300  433.1633  435.07
6   2024-01-10  435.77  438.7350  435.4800  437.94
7   2024-01-11  438.94  439.3550  434.0000  437.79
8   2024-01-12  439.11  439.8000  436.7300  437.99
9   2024-01-16  436.79  437.9800  434.7400  436.50
10  2024-01-17  433.60  434.4800  431.8100  434.07
11  2024-01-18  435.58  438.3900  434.1600  437.93
12  2024-01-19  439.00  443.6100  438.3100  443.29
13  2024-01-22  444.77  445.9200  443.6800  444.30
14  2024-01-23  444.83  445.8200  443.7800  445.62
15  2024-01-24  448.32  449.1896  445.7500  445.99
16  2024-01-25  448.14  448.7600

In [51]:
url = f"https://api.tiingo.com/iex/{'brk-b'}/prices?token={TIINGO_API_TOKEN}"
response = requests.get(url)
data = response.json()
data

{'detail': 'Error: You have run over your hourly request allocation. Please upgrade at https://api.tiingo.com/pricing to have your limits increased.'}

In [38]:
import os
import requests
import pandas as pd

# Define your Tiingo API token
TIINGO_API_TOKEN = "7420f870dae9f398dc43b1cd54378032f115265e"

def get_latest_price_realtime_tiingo(stock_name: str):
    stock_name = stock_name.upper()
    url = f"https://api.tiingo.com/iex/{stock_name}/prices?token={TIINGO_API_TOKEN}"
    response = requests.get(url)
    data = response.json()
    latest_price = data[-1]['close']
    return latest_price

def get_historical_prices_tiingo(
    stock_name: str,
    start: str,
    end: str):
    stock_name = stock_name.upper()
    url = f"https://api.tiingo.com/tiingo/daily/{stock_name}/prices?startDate={start}&endDate={end}&token={TIINGO_API_TOKEN}"
    response = requests.get(url)
    data = response.json()
    prices = []
    for item in data:
        prices.append(item)
    df_stock = pd.DataFrame(prices)
    df_stock.columns = df_stock.columns.str.lower()
    df_stock['date'] = pd.to_datetime(df_stock['date'])
    return df_stock

if __name__ == "__main__":
    # (1) Get the latest price for VOO in real-time
    latest_voo_price_realtime_tiingo = get_latest_price_realtime_tiingo("VOO")
    print(f"Latest price for VOO in real-time (Tiingo): ${latest_voo_price_realtime_tiingo:.2f}")

    # (2) Get historical daily prices for VOO from 2024-01-01 to 2024-01-31
    historical_voo_prices_tiingo = get_historical_prices_tiingo("VOO", "2024-01-01", "2024-01-31")
    print("\nHistorical prices for VOO (Tiingo):")
    print(historical_voo_prices_tiingo)

Latest price for VOO in real-time (Tiingo): $474.79

Historical prices for VOO (Tiingo):
                        date   close      high       low    open   volume  \
0  2024-01-02 00:00:00+00:00  434.01  435.2700  432.3600  433.91  9076041   
1  2024-01-03 00:00:00+00:00  430.79  433.0000  430.2500  432.32  4805049   
2  2024-01-04 00:00:00+00:00  429.43  432.7800  429.2100  430.33  3945259   
3  2024-01-05 00:00:00+00:00  429.98  432.3000  428.6400  429.59  5693235   
4  2024-01-08 00:00:00+00:00  436.13  436.2700  430.3700  430.45  7222362   
5  2024-01-09 00:00:00+00:00  435.07  436.4300  433.1633  433.62  7939567   
6  2024-01-10 00:00:00+00:00  437.94  438.7350  435.4800  435.77  4278841   
7  2024-01-11 00:00:00+00:00  437.79  439.3550  434.0000  438.94  4796587   
8  2024-01-12 00:00:00+00:00  437.99  439.8000  436.7300  439.11  5182669   
9  2024-01-16 00:00:00+00:00  436.50  437.9800  434.7400  436.79  5730839   
10 2024-01-17 00:00:00+00:00  434.07  434.4800  431.8100  433.60

In [41]:
from dateutil.relativedelta import relativedelta

df = get_historical_prices_tiingo('brk-b',
        	start=(datetime.today() - relativedelta(days=200)).strftime('%Y-%m-%d'),
        	end=datetime.today().strftime('%Y-%m-%d')
        	)
df

,date,close,high,low,open,volume,adjclose,adjhigh,adjlow,adjopen,adjvolume,divcash,splitfactor
0,2023-10-19 00:00:00+00:00,338.66,342.69,338.45,340.31,2745685,338.66,342.69,338.45,340.31,2745685,0.0,1.0
1,2023-10-20 00:00:00+00:00,335.86,340.00,334.35,338.15,3472804,335.86,340.00,334.35,338.15,3472804,0.0,1.0
2,2023-10-23 00:00:00+00:00,336.84,338.88,333.49,334.07,2795170,336.84,338.88,333.49,334.07,2795170,0.0,1.0
3,2023-10-24 00:00:00+00:00,338.63,339.85,337.77,338.18,2356386,338.63,339.85,337.77,338.18,2356386,0.0,1.0
4,2023-10-25 00:00:00+00:00,336.90,339.62,336.55,338.59,2624711,336.90,339.62,336.55,338.59,2624711,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2024-04-29 00:00:00+00:00,400.96,404.87,400.08,402.66,2365421,400.96,404.87,400.08,402.66,2365421,0.0,1.0
132,2024-04-30 00:00:00+00:00,396.73,400.60,396.37,400.58,3074765,396.73,400.60,396.37,400.58,3074765,0.0,1.0
133,2024-05-01 00:00:00+00:00,398.58,402.32,396.35,396.61,2606198,398.58,402.32,396.35,396.61,2606198,0.0,1.0
134,2024-05-02 00:00:00+00:00,400.60,401.56,397.88,401.00,2679740,400.60,401.56,397.88,401.00,2679740,0.0,1.0


In [42]:
last_19day_price = df['close'][-19:]
last_19day_price

117    414.69
118    409.11
119    407.61
120    403.26
121    400.26
122    396.92
123    397.74
124    399.89
125    405.08
126    408.78
127    408.74
128    405.95
129    404.91
130    402.10
131    400.96
132    396.73
133    398.58
134    400.60
135    400.87
Name: close, dtype: float64